# WORDLE Breaker

### https://www.powerlanguage.co.uk/wordle/  

In [1]:
import re
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def special_match(pattern, strg):
    result = re.match(pattern, strg)
    return result is not None


In [52]:
def break_wordle(attempts, word_lengh, dictionary):
   
    
    correct_pos_list = [] 
    incorrect_pos_list = [] 
    exist_in_list  = []
    not_exist_list = ["."]
    not_recognized_word_list = []    
    try_list = []
    regex_list = []
    
    correct_pos_list    = ["a-z"] * word_lengh
    incorrect_pos_list  = ["a-z"] * word_lengh

    correct_pos_rex   = "^"+ "".join( "[" + str(x) +"]" for x in correct_pos_list ) +"$"
    incorrect_pos_rex = "^"+ "".join( "[" + str(x) +"]" for x in incorrect_pos_list ) +"$"
    exist_in_rex      = "^.*" + "".join( "(?=.*[" + str(x) +"])" for x in exist_in_list )  +".*"
    not_exist_rex     = "^.*(?=.*[" + "".join(str(x) for x in not_exist_list )  +"]).*"
    

    for i in range(attempts):        
        print("\n###############################################################################################")
        print(f'''Attempt {i+1}:''')
        
        print(f'''\nRegex Used:
        Correct Pos:          {correct_pos_rex}                    
        InCorrect Pos:        {incorrect_pos_rex}                    
        Contain:              {exist_in_rex}
        Does not Contain:     {not_exist_rex}
        Not Recognized words: {not_recognized_word_list}
        ''')

        if (i==0):
            newWordList = dictionary[dictionary[0].str.contains(correct_pos_rex)==True]
        else:
            newWordList = newWordList[newWordList[0].str.contains(correct_pos_rex)==True]
        newWordList = newWordList[newWordList[0].str.contains(incorrect_pos_rex)==True]
        newWordList = newWordList[newWordList[0].str.contains(exist_in_rex)==True]
        newWordList = newWordList[newWordList[0].str.contains(not_exist_rex)==False]
        newWordList = newWordList[newWordList[0].isin(not_recognized_word_list)==False]
        
        regex_list.append([correct_pos_rex, incorrect_pos_rex, exist_in_rex, not_exist_rex])
       
        if (i==0):
            newWordList["var"]=newWordList.apply(lambda x: 1-(sum([1 for y in x[0] if (x[0].count(y)-1) > 0]) / len(x[0])) , axis=1)
            newWordList["weight"]=newWordList.apply(lambda x: sum([1.5 for y in x[0] if y in ['e', 'a', 'i', 'o']]) + sum([1 for y in x[0] if y in ['t', 'n', 's', 'h', 'r']]), axis=1)
        
        newWordList =newWordList.sort_values(["var", "weight"], ascending=False)

                
        if (len(newWordList) > 0 ):
            str_try = newWordList.head(1)[0].item()
        else:
            print("\nDictionary empty. You lose")
            return
       
        if (len(newWordList) > 1 ):
            print(f'''Try the word: {str_try} (1 of {len(newWordList)} or {"{:.2f}".format(((1/len(newWordList))*100))}% of success)''')
        else:
            print(f'''The word (probably) is: {str_try} (1 of {len(newWordList)} or {"{:.2f}".format(((1/len(newWordList))*100))}% of success)''')

        resp = ""
        while len(resp) != word_lengh or special_match(f'''^[r|g|y]{{{word_lengh}}}$''',resp)==False:
            resp = input(f'''Enter {word_lengh} char sequence (no spaces)
                            "g" - green 
                            "y" - yellow 
                            "r" - gray
                            'x' - not recognized word: ''')
            if (resp=="x"):
                not_recognized_word_list.append(str_try)
                newWordList = newWordList[newWordList[0].isin(not_recognized_word_list)==False]
                str_try = newWordList.head(1)[0].item()                
                print(f'''\nNew word on Attempt {i+1}''')
                print(f'''Try the word: {str_try} (1 of {len(newWordList)} or {"{:.2f}".format(((1/len(newWordList))*100))}% of success)''')
            else:
                if (str_try not in try_list):
                    try_list.append(str_try)
                
            
        if (resp == 'g' * word_lengh):
            print("\nCongrats, you won!")
            for x in range(len(try_list)):
                print(f'''
Word: {try_list[x]}
Regex Used:
    Correct Pos:     {regex_list[x][0]}                    
    InCorrect Pos:   {regex_list[x][1]}                    
    Contain:         {regex_list[x][2]}
    Does not Contain:{regex_list[x][3]}
    List of existing words tried:
        {not_recognized_word_list} ''')            
            return
        
        for i in range(len(resp)):   
            if resp[i] == 'g':
                correct_pos_list[i] = str_try[i]
            if resp[i] == 'r':
                if (not_exist_list[0] == "."):
                    not_exist_list[0] = str_try[i]
                else:
                    not_exist_list.append(str_try[i])
            if resp[i] == 'y':
                if (incorrect_pos_list[i] == "a-z"):
                    incorrect_pos_list[i] = "^" + str_try[i]
                else:
                    incorrect_pos_list[i] = incorrect_pos_list[i] + str_try[i]                     
                
                if (str_try[i] not in exist_in_list):
                    exist_in_list.append(str_try[i])
        
        temp_list = not_exist_list.copy()
        for i in range(len(not_exist_list)):
            if (not_exist_list[i] in correct_pos_list):
                temp_list.remove(not_exist_list[i])
                y =  correct_pos_list.index(not_exist_list[i])
                for x in range(len(incorrect_pos_list)):              
                    if (x != y):
                        if (incorrect_pos_list[x] == "a-z"):
                            incorrect_pos_list[x] = "^" + not_exist_list[i]
                        else:
                            incorrect_pos_list[x] = incorrect_pos_list[x] + not_exist_list[i]                     
                
                    
        
        not_exist_list = temp_list.copy()
        
        
        correct_pos_rex   = "^"+ "".join( "[" + str(x) +"]" for x in correct_pos_list ) +"$"
        incorrect_pos_rex = "^"+ "".join( "[" + str(x) +"]" for x in incorrect_pos_list ) +"$"
        exist_in_rex      = "^.*" + "".join( "(?=.*[" + str(x) +"])" for x in exist_in_list )  +".*"
        not_exist_rex     = "^.*(?=.*[" + "".join(str(x) for x in not_exist_list )  +"]).*"

    print("\nSorry, you lose")
        


In [4]:
words = pd.read_csv('https://github.com/dwyl/english-words/raw/master/words_alpha.txt', header=None)
    
#words = pd.read_csv('https://raw.githubusercontent.com/IlyaSemenov/wikipedia-word-frequency/master/results/enwiki-20190320-words-frequency.txt', sep=' ', header=None)


In [ ]:
break_wordle(6,5,words)


###############################################################################################
Attempt 1:

Regex Used:
        Correct Pos:          ^[a-z][a-z][a-z][a-z][a-z]$                    
        InCorrect Pos:        ^[a-z][a-z][a-z][a-z][a-z]$                    
        Contain:              ^.*.*
        Does not Contain:     ^.*(?=.*[.]).*
        Not Recognized words: []
        
Try the word: aeons (1 of 15918 or 0.01% of success)


Enter 5 char sequence (no spaces)
                            "g" - green 
                            "y" - yellow 
                            "r" - gray
                            'x' - not recognized word:  yryrr



###############################################################################################
Attempt 2:

Regex Used:
        Correct Pos:          ^[a-z][a-z][a-z][a-z][a-z]$                    
        InCorrect Pos:        ^[^a][a-z][^o][a-z][a-z]$                    
        Contain:              ^.*(?=.*[a])(?=.*[o]).*
        Does not Contain:     ^.*(?=.*[ens]).*
        Not Recognized words: []
        
Try the word: ratio (1 of 432 or 0.23% of success)


Enter 5 char sequence (no spaces)
                            "g" - green 
                            "y" - yellow 
                            "r" - gray
                            'x' - not recognized word:  ygrry



###############################################################################################
Attempt 3:

Regex Used:
        Correct Pos:          ^[a-z][a][a-z][a-z][a-z]$                    
        InCorrect Pos:        ^[^ar][a-z][^o][a-z][^o]$                    
        Contain:              ^.*(?=.*[a])(?=.*[o])(?=.*[r]).*
        Does not Contain:     ^.*(?=.*[ensti]).*
        Not Recognized words: []
        
Try the word: balor (1 of 19 or 5.26% of success)


...


New word on Attempt 3
Try the word: calor (1 of 18 or 5.56% of success)


Enter 5 char sequence (no spaces)
                            "g" - green 
                            "y" - yellow 
                            "r" - gray
                            'x' - not recognized word:  x



New word on Attempt 3
Try the word: carob (1 of 17 or 5.88% of success)


Enter 5 char sequence (no spaces)
                            "g" - green 
                            "y" - yellow 
                            "r" - gray
                            'x' - not recognized word:  rgygr



###############################################################################################
Attempt 4:

Regex Used:
        Correct Pos:          ^[a-z][a][a-z][o][a-z]$                    
        InCorrect Pos:        ^[^ar][a-z][^or][a-z][^o]$                    
        Contain:              ^.*(?=.*[a])(?=.*[o])(?=.*[r]).*
        Does not Contain:     ^.*(?=.*[ensticb]).*
        Not Recognized words: ['balor', 'calor']
        
Try the word: favor (1 of 8 or 12.50% of success)


<generator object <genexpr> at 0x0000022FB66885F0>
1
2
11
22
